In [1]:
import numpy as np
import pandas as pd
import keras
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('/Users/amirhossein/Desktop/all/Uni/MS/rahnama/recommander/data/data1/recommender/train_interactions.csv')

In [3]:
train_df.drop(columns=['interaction_min', 'delivery_min','interaction_dow','interaction_hour','delivery_dow','delivery_hour'], inplace=True)

In [4]:
test_df = pd.read_csv('/Users/amirhossein/Desktop/all/Uni/MS/rahnama/recommander/data/data1/recommender/test.csv')

In [5]:
users_df=pd.read_csv("/Users/amirhossein/Desktop/all/Uni/MS/rahnama/recommander/data/data1/recommender/user_sparse_1.csv")
users_df.drop(columns=['I95', 'I3','I2','I4','I5','I94','I6','I93','I7','I8','I92','I9','I96','I1','I10','I91','I11','I90','I12','I89','I13','I88'],inplace=True)


In [6]:
ids=users_df['user_id']


In [7]:
users_df.fillna(users_df.median(),inplace=True)

In [8]:
users_df.drop(columns=['user_id'],inplace=True)
#print(data)

In [9]:
print("salam")

salam


In [10]:
import numpy as np
from sklearn.decomposition import PCA


pca = PCA(n_components=10)
a=pca.fit(users_df)

In [11]:
p=a.transform(users_df)

In [12]:
users_df=p

In [13]:
print(len(users_df))

3933884


In [14]:
r=users_df[0]

In [15]:
print(r)

[-192.92718463  -10.51290932   -1.04324045   24.65220863   26.81856802
   -2.49806104    7.41294813   -7.04477383   21.89276378   -5.59895686]


In [16]:
#data=pd.read_csv('/Users/amirhossein/Desktop/all/Uni/MS/rahnama/recommander/data/data1/recommender/user_sparse_1.csv')
#user2features = {}
#for i in range(0,len(users_df)):
#ids=data['user_id']

    
    

In [17]:
print(len(ids))

3933884


In [18]:
user2features = {}

for i in range(0,len(ids)):
    user2features[ids[i]]=users_df[i]
    
    
    

In [37]:
print(len(user2features))

3933884


In [20]:
print((user2features[8226308]))

[-192.92718463  -10.51290932   -1.04324045   24.65220863   26.81856802
   -2.49806104    7.41294813   -7.04477383   21.89276378   -5.59895686]


In [21]:
notif_df = pd.read_csv('/Users/amirhossein/Desktop/all/Uni/MS/rahnama/recommander/data/data1/recommender/notifs_corrected.csv')
notif_df.head()

,notif_id,day_of_week,hour,minute,category,text
0,568156,6,17,30,7,135 37 8 39 105 1503 1504 25 1 161 35 213
1,567822,6,15,46,7,72 104 1 139 93 95 137 66 537 332 139 93 495 1...
2,567810,6,15,45,7,72 104 163 86 72 311 1712 1
3,567886,6,15,54,5,198 221 1426 538 1713 54 3 27 1714 716 38 145 ...
4,568058,6,16,13,7,72 104 1 139 93 95 137 66 537 332 139 93 495 1...


In [43]:
notif2features = {}
notifs_vecs = []
notif_ids = []
for index, row in tqdm(notif_df.iterrows()):
    if type(row.text) == str:
        words = [int(word) for word in row.text.split(' ')]
        vec = np.zeros(2000)
        vec[words] = 1
        notif2features[row.notif_id] = vec
        notifs_vecs.append(vec)
        notif_ids.append(row.notif_id)
notifs_vecs = np.array(notifs_vecs)
print((len)(notifs_vecs[0]))

6347it [00:00, 6471.08it/s]


2000


In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
notifs_small_vecs = pca.fit_transform(notifs_vecs)
for idx,notif_id in enumerate(notif_ids):
    notif2features[notif_id] = notifs_small_vecs[idx]

In [24]:
notif2dow = {}
notif2hour = {}
for index, row in notif_df.iterrows():
    notif2hour[row.notif_id] = row.hour

In [25]:
notif_df_filtered = notif_df[(notif_df.category == 3)]
notif_df_filtered.shape

(915, 6)

In [26]:
notifs_selected = notif_df_filtered.notif_id.unique()
#notifs_selected.shape

In [29]:
test_df[(test_df.notif_id.isin(notifs_selected))].shape
#print((len)(notifs_selected))

915


In [30]:
train_df.head()

,user_id,notif_id,interaction
0,654408,468552,0
1,9272634,517721,0
2,380089,519842,0
3,2586969,410941,0
4,491160,463087,0


In [32]:
train_df_filtered = train_df[train_df.notif_id.isin(notifs_selected)].copy()
train_df_filtered.shape

(13148959, 3)

In [33]:
train_df_filtered.head()

,user_id,notif_id,interaction
8,1644236,517398,0
14,6726471,414617,0
18,10272948,463254,0
19,5245236,521158,1
37,429251,463914,0


In [34]:
USERS_FEATURES =10
NOTIFS_FEATURES = 10

In [35]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, batch_size=32, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.data = data
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.data.shape[0] // self.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        Y = self.data[indexes,2]
        _X = self.data[indexes]
        X = np.zeros([_X.shape[0], USERS_FEATURES + NOTIFS_FEATURES])
        try:
            X[:,0:10] = [user2features[user_id] for user_id in _X[:,0]]
        except:
             X[:,0:10] = [0,0,0,0,0,0,0,0,0,0]
        X[:,10:20] = [notif2features[notif_id] for notif_id in _X[:,1]]#inja
        weight = np.array([0.05 if y == 0 else 0.95 for y in Y])
        return X, Y, weight

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.data.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)



In [36]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
# Datasets
# partition = # IDs
# labels = # Labels

# Generators
training_generator = DataGenerator(train_df_filtered.values, shuffle=True, batch_size=32)
validation_generator = DataGenerator(train_df_filtered.values, shuffle=True, batch_size=32)

input_dim = USERS_FEATURES + NOTIFS_FEATURES
model = Sequential([

    Dense(32, input_shape=(input_dim,)),
    Activation('relu'),
    Dense(16, input_shape=(input_dim,)),
    Activation('relu'),
    Dense(1),
    Activation('softmax'),
])

# For a mean squared error regression problem
model.compile(optimizer='adam', loss='binary_crossentropy')


# Design model
# model = Sequential()
# [...] # Architecture
# model.compile()

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=7,
#                     class_weight={0: 0.05, 1: 0.95},
                    epochs=100)



Epoch 1/100
410894/410904 [============================>.] - ETA: 0s - loss: 0.7579

Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-5:
Process ForkPoolWorker-7:
Process ForkPoolWorker-6:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self.

KeyboardInterrupt: 